# How to read hdf4eos with 
- pyhdf
- gdal
- rasterio

Note: https://github.com/rasterio/rasterio/issues/2026
--> Need to use the system's gdal. I.e. do something like:
```
pip3 uninstall rasterio
sudo apt install libgdal-dev/focal
pip3 install rasterio --no-binary rasterio
```

In [2]:
import pyhdf.SD
from osgeo import gdal
import rasterio

In [5]:
tile_name = '/tablespace/spires/mod09ga/MOD09GA.A2021001.h08v05.006.2021003022347.hdf'
group = 'MODIS_Grid_500m_2D'
ds = 'sur_refl_b01_1'
ds_trunk = 'HDF4_EOS:EOS_GRID:{file}:{group}:{ds}'
ds_string = ds_trunk.format(file=tile_name, group=group, ds=ds)

# Rasterio

In [4]:
data = rasterio.open(ds_string) 
left = data.bounds.left
right = data.bounds.right
width = data.width

bottom = data.bounds.bottom
top = data.bounds.top
height = data.height

r = data.crs.to_dict()['R'] # Earth Radius
r

data.read()

array([[[   0,   28,   28, ..., 1805,  918, 1332],
        [  -1,   35,   35, ..., 1389, 1595, 1419],
        [  -7,   -7,   -7, ..., 1403, 2189, 3363],
        ...,
        [ 758,  758,  778, ..., 6201, 6201, 6421],
        [ 758,  778,  778, ..., 6201, 6378, 6378],
        [ 709,  709,  820, ..., 6104, 6104, 5855]]], dtype=int16)

# pyhdf

In [26]:
tile_sd = pyhdf.SD.SD(tile_name)
sd = tile_sd.select('sur_refl_b01_1')
#sd.attr('scale_factor').get()
attributes = sd.attributes()
attributes

{'long_name': '500m Surface Reflectance Band 1 - first layer',
 'units': 'reflectance',
 'valid_range': [-100, 16000],
 '_FillValue': -28672,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5,
 'scale_factor': 10000.0,
 'scale_factor_err': 0.0,
 'Nadir Data Resolution': '500m'}

In [52]:
import numpy
data = sd.get()

d = numpy.ma.array(data, mask=(data==-28672))
d / sd.attributes()['scale_factor']

masked_array(
  data=[[0.0004, 0.0036, 0.0036, ..., 0.1752, 0.0849, 0.1303],
        [0.0005, 0.0039, 0.0039, ..., 0.133, 0.1568, 0.1391],
        [-0.0002, -0.0002, -0.0002, ..., 0.1376, 0.218, 0.3349],
        ...,
        [0.0754, 0.0754, 0.0774, ..., 0.6179, 0.6179, 0.6399],
        [0.0754, 0.0774, 0.0774, ..., 0.6179, 0.6355, 0.6355],
        [0.0702, 0.0702, 0.0817, ..., 0.6082, 0.6082, 0.5835]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],
  fill_value=1e+20)

In [ ]:
# MOD09

# GDAL

In [6]:

tile_gdal = gdal.Open(ds_string, gdal.GA_ReadOnly)
metadata = tile_gdal.GetMetadata()

In [7]:
trans = tile_gdal.GetGeoTransform()
tile_gdal.GetProjection()
tile_gdal.ReadAsArray()

array([[   0,   28,   28, ..., 1805,  918, 1332],
       [  -1,   35,   35, ..., 1389, 1595, 1419],
       [  -7,   -7,   -7, ..., 1403, 2189, 3363],
       ...,
       [ 758,  758,  778, ..., 6201, 6201, 6421],
       [ 758,  778,  778, ..., 6201, 6378, 6378],
       [ 709,  709,  820, ..., 6104, 6104, 5855]], dtype=int16)

In [8]:
width = tile_gdal.RasterXSize
height = tile_gdal.RasterYSize

y_res = trans[5]
x_res = trans[1]

left = trans[0]
right = left + tile_gdal.RasterXSize*width
top = trans[3]
bottom = top + tile_gdal.RasterYSize*height